In [ ]:
import os
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import pytz
from datetime import datetime, timedelta
from time import sleep
import netCDF4 as nc
import h5netcdf
import glob
# import cf_xarray as cfxr
import shapely
# import xesmf as xe
import geopandas as gpd
import numpy as np
import cartopy.feature as cfeature
import cartopy.crs as ccrs   
from matplotlib.pyplot import figure

# ERA5

In [ ]:
df = xr.open_dataset('/home/mendozaemir/research/remote-sensing-landuse/data/ERA5_reanalysis_nairobi_d01')
df

lat=df.latitude
lon=df.longitude
precip=df.tp
print(precip)

precip_sum = precip.sum(dim='time')
print(precip_sum)


In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())

cf = plt.contourf(lon, lat, precip_sum*1000, cmap='jet')
cb = plt.colorbar(cf, fraction=0.0235, pad=0.05)
cb.set_label('mm', fontsize=15, rotation=0, loc='bottom')

ax.add_feature(cfeature.COASTLINE)

plt.show()

# CMORPH

In [ ]:
cmorph_list = []
for filepath in glob.iglob("/home/mendozaemir/research/remote-sensing-landuse/data/cmorph/*.nc"):
    ncfile = xr.open_dataset(filepath, chunks = {'time': 10})
    cmorph_list.append(ncfile)


In [ ]:
lat = cmorph_list[5].lat
lon = cmorph_list[5].lon
time = cmorph_list[23].time
print(lat)
print(lon)
cmorph = 0
for ds in cmorph_list:
    cmorph_hr = ds.cmorph.sum(axis=0)
    cmorph += cmorph_hr.values



# NAIROBI, KENYA

In [ ]:
# Domain 1
# 36.749, -1.282
# 30.136 ~ 43.362, -7.869 ~ 5.323

# 6.613, -6.587~+6.605

# Domain 3
# 36.749, -1.282
# 35.987 ~ 37.323, -2.044 ~ -0.708
# -0.762 ~ +0.574, -0.762 ~ +0.574


ax = plt.axes(projection=ccrs.PlateCarree())

lonbounds = (lon > 35.987) & (lon < 37.323)
latbounds = (lat > -2.044) & (lat < -0.708)
lonrange = lon[lonbounds]
latrange = lat[latbounds]
lonindices = np.where(lonbounds)[0]
latindices = np.where(latbounds)[0]

# code to control scale of contourf -> levels=np.linspace(0,20)
cf = plt.contourf(lonrange, latrange, cmorph[latindices, lonindices[:, np.newaxis]].T, cmap='jet')
cb = plt.colorbar(cf, fraction=0.0235, pad=0.05)
cb.set_label('mm', fontsize=15, rotation=0, loc='bottom')

ax.add_feature(cfeature.COASTLINE)

plt.show()

# Lagos, Nigeria

In [ ]:
# Lagos Nigeria: 3.351209, 6.421560

ax = plt.axes(projection=ccrs.PlateCarree())


lonbounds = (lon > -3.262) & (lon < 9.964)
latbounds = (lat > -0.165) & (lat < 13.027)
lonrange = lon[lonbounds]
latrange = lat[latbounds]
lonindices = np.where(lonbounds)[0]
latindices = np.where(latbounds)[0]
cf = plt.contourf(lonrange, latrange, cmorph[latindices,lonindices[:, np.newaxis]].T, cmap='jet')
cb = plt.colorbar(cf, fraction=0.0235, pad=0.05)
cb.set_label('mm', fontsize=15, rotation=0, loc='bottom')

ax.add_feature(cfeature.COASTLINE)

plt.show()

# Accra, Ghana

In [ ]:
# DOMAIN 3 # -0.762 ~ +0.574, -0.762 ~ +0.574
# Accra Ghana: -0.1870, 5.6037
# 6.613, -6.587~+6.605
# -6.8 ~ +6.426, -0.9833 ~ 12.2087

# Malabon PH: 120.960334, 14.665835
# DOMAIN 1 114.36 ~ 127.56, 8.08 ~ 21.28
# DOMAIN 3 
ax = plt.axes(projection=ccrs.PlateCarree())

# For Ghana, longitude is from ranges 0 to 359. might need to change how to capture areas nearing the edge values
lonbounds = (lon > 120.20) & (lon < 121.53)
latbounds = (lat > 13.908) & (lat < 15.244)
lonrange = lon[lonbounds]
latrange = lat[latbounds]
lonindices = np.where(lonbounds)[0]
latindices = np.where(latbounds)[0]

# code to control scale of contourf -> levels=np.linspace(0,20)
cf = plt.contourf(lonrange, latrange, cmorph[latindices,lonindices[:, np.newaxis]].T, cmap='jet')
cb = plt.colorbar(cf, fraction=0.0235, pad=0.05)
cb.set_label('mm', fontsize=15, rotation=0, loc='bottom')

ax.add_feature(cfeature.COASTLINE)

plt.show()

# WRF RESULTS

In [ ]:
wrf_file = '/home/mendozaemir/research/remote-sensing-landuse/data/wrfout_d01_2021-04-01_00:00:00'
df_wrf = xr.open_dataset(wrf_file)
lat_wrf=df_wrf.XLAT
lon_wrf=df_wrf.XLONG
precip=df_wrf.RAINNC + df_wrf.RAINC 
print(precip[714].shape)

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())

cf = plt.contourf(lon_wrf[0], lat_wrf[0], precip[714], cmap='jet')
cb = plt.colorbar(cf, fraction=0.0235, pad=0.05)
cb.set_label('mm', fontsize=15, rotation=0, loc='bottom')

ax.add_feature(cfeature.COASTLINE)

plt.show()

In [ ]:
# try to download contourf plot as tif to visualize in qgis. currently dismissed because rainc value
# in nairobi d03 is 0, but i eventually need the rainc+rainnc values
for c in cf.collections:
    c.set_rasterized(True)

plt.savefig('do3_precip.tif', dpi=100)

In [ ]:
def generate_times(df):
    times = df.variables['XTIME']
    times = nc.num2date(times[:],times.units,only_use_cftime_datetimes=False)
    return times

def timeseries(ifil,axs,label):
    var1 = df.variables['RAINNC'][:,:,:]
    var2 = df.variables['RAINC'][:,:,:]
    var = var1+var2
    var = np.nanmean(var,axis=(1,2))
    var = np.roll(var,-1,axis=0)-var
    times = generate_times(df)
    axs.plot(times[:-1],var[:-1],label=label)
    return axs


# Maximum Monthly Rainfall by hour

In [ ]:
precip_frm = np.nanmax(precip,axis=(1,2))
precip_frm = np.roll(precip_frm,-1,axis=0)-precip_frm
print(precip_frm.max())
times = df_wrf.XTIME

# enddate specifies the date range to be plotted (hours since starting point).
enddate = 714
fig, ax = plt.subplots() 
ax.plot(times[:enddate],precip_frm[:enddate])
fig.autofmt_xdate()

# Average Monthly

In [ ]:
precip_frm = np.nanmean(precip,axis=(1,2))
precip_frm = np.roll(precip_frm,-1,axis=0)-precip_frm
print(precip_frm.max())
print(precip_frm[:714].mean())
times = df_wrf.XTIME

# enddate specifies the date range to be plotted (hours since starting point).
enddate = 714
fig, ax = plt.subplots() 
ax.plot(times[:enddate],precip_frm[:enddate])
fig.autofmt_xdate()

# Maximum Daily Rainfall

In [ ]:
precip_frm = np.nanmax(precip,axis=(1,2))
precip_frm = np.roll(precip_frm,-1,axis=0)-precip_frm
precip_split = np.array_split(precip_frm, 30)

daily_max_list = []
for day in precip_split:
    daily_max = day.max()
    daily_max_list.append(daily_max)

times = df_wrf.XTIME
# enddate specifies the date range to be plotted (hours since starting point).
enddate = 714
fig, ax = plt.subplots() 
ax.plot(times[0:-1:24],daily_max_list)
fig.autofmt_xdate()
print(max(daily_max_list))
